# 🧩 Reasoning with LLMs: Solving the Game of 24

Welcome to this module on **reasoning frameworks for Large Language Models**.  
In this notebook, we’ll explore how different prompting and inference strategies influence an LLM’s ability to solve a classic math reasoning task: the **Game of 24**.

You’ll work with:

- A dataset of 24-puzzles  
- Verification tools to check whether a solution is valid  
- Groq as our inference engine for fast, iterative reasoning  
- Several reasoning frameworks, including:  
  - **Few-Shot Prompting**  
  - **Chain of Thought (CoT)**  
  - **ReAct (Reason + Act)**  
  - **Tree of Thoughts (ToT) — BFS variant**

As you progress, you’ll implement each framework yourself and compare how they perform on the same underlying task.  
By the end, you’ll have hands-on experience with multiple LLM reasoning paradigms and a deeper understanding of how structured prompting can guide models toward more reliable problem-solving.

Let’s get started!

## 📘 The Game of 24 Dataset

The **Game of 24** is a classic arithmetic puzzle: Given **four numbers**, your goal is to combine them using the operations **+**, **−**, **×**, and **÷** (and parentheses) to make the value **24**.

For example:  
- Numbers: `4 7 8 8`  
- One solution: `(7 - 4) × 8 = 24`

### 🗂️ What’s in the Dataset?

The *Game of 24 dataset* consists of many such **four-number puzzles**, typically drawn from the range 1–9. Each entry commonly includes:
- The **four digits** that make up the puzzle  
- Some descriptive statistics drawn from *[https://www.4nums.com/](https://www.4nums.com/)* (this is where the dataset was extracted from)

Not all sets of numbers can make 24—part, but all entries in this dataset do!

### 🔍 In This Notebook

The dataset has been downloaded the code to load it is given below. If you're curious feel free to explore the dataset on your own.

In [ ]:
import pandas as pd
from datasets import Dataset


df = pd.read_csv("dataset_game24.csv.gz", compression="gzip")
display(df.head())
dataset = Dataset.from_pandas(df)

**`Discussion:`** Solve the first two puzzles on your own to get some intuition of the game.
- 1 1 4 6
- 1 1 11 11

\[Your Answer\]

### 🧪 Verifying Student Solutions Programmatically

To help you check whether a proposed solution to a Game of 24 puzzle is valid, we provide a small evaluation utility.  
You won’t need to understand every detail of the code right away, just how to use it.

#### 🔍 What the Verifier Does

The evaluation functions perform three main checks:

1. **Is the solution “final”?**  
   The code checks whether your sequence of steps ends in a legitimate final state — that is, you’ve produced a single expression without leftover numbers.

2. **Did you use the correct numbers?**  
   Your final expression must use *exactly* the four numbers from the puzzle, no more and no fewer.

3. **Does the expression equal 24?**  
   Using SymPy, the tool evaluates your final expression safely and determines whether it results in 24.

#### 🧩 How You’ll Use It

You’ll call:

```python
is_terminal, reward = evaluate(steps, puzzle, current_state)
```

**Where:**

- **`steps`** is a list of strings describing the steps the student (or your algorithm) took  
- **`puzzle`** is the original four-number puzzle as a string  
- **`current_state`** the remaining (unsued) numbers

**The function returns:**

- **`is_terminal`** whether the attempt reached a final state  
- **`reward`**
  - `1.0` if the final expression correctly evaluates to 24  
  - `0.0` otherwise  


### ✔️ Example

```python
steps = ["1 * 1 = 1 (left: 1 3 8)", "1 * 3 = 3 (left: 3 8)", "3 * 8 = 24 (left: 24)", "Answer: (((1 * 1) * 3) * 8) = 24"]
puzzle = "1 1 3 8"
current_state = "24"

evaluate(steps, puzzle, current_state)
# → (True, 1.0)

In [ ]:
import re
from sympy import simplify
from typing import Tuple, List


def is_final(steps: List[str], current_state: str) -> bool:
    """
    Determine whether the transition sequence ends in a final state.

    A state is considered final if:
    - There is at least one step
    - The last step does not contain the word "left"
    - The current_state string contains only one token
    """
    if not steps:
        return False

    expression = steps[-1]

    if "left" in expression:
        return False

    # current_state should be a single token (no spaces)
    if len(current_state.split()) > 1:
        return False

    return True


def evaluate(
    steps: List[str],
    puzzle: str,
    current_state: str
) -> Tuple[bool, float]:
    """
    Evaluate the puzzle result in a fully self-contained way.

    Returns:
        (is_terminal_state: bool, reward: float)

    Reward rules:
    - If final and expression uses incorrect numbers → reward 0
    - If final and evaluates to 24 → reward 1
    - Otherwise → reward 0
    """
    final = is_final(steps, current_state)

    if not final or not steps or not steps[-1]:
        return False, 0.0

    # Extract the expression from the final step
    expression = steps[-1].lower().replace("answer: ", "")
    expression = expression.split("=")[0]

    # Numbers used in the player's expression
    numbers_used = re.findall(r"\d+", expression)

    # Numbers available in the original puzzle
    puzzle_numbers = re.findall(r"\d+", puzzle)

    # Check whether they used exactly the correct numbers
    if sorted(numbers_used) != sorted(puzzle_numbers):
        print("Incorrect numbers used")
        return True, 0.0

    # Safely evaluate expression
    try:
        correct = simplify(expression) == 24
        print("Expression is correct: ", bool(correct))
        return True, float(correct)
    except Exception:
        print("Error evaluating expression")
        return True, 0.0

# Example usage
steps = ["1 * 1 = 1 (left: 1 3 8)", "1 * 3 = 3 (left: 3 8)", "3 * 8 = 24 (left: 24)", "Answer: (((1 * 1) * 3) * 8) = 24"]
puzzle = "1 1 3 8"
current_state = "24"

evaluate(steps, puzzle, current_state)

## 🚀 Inference Engine: Using Groq

For all of our **inference-based reasoning steps**, we’ll be using **Groq** as our model host and execution engine.  

If you haven’t used Groq before, don’t worry, the setup and usage should already be familiar from earlier exercises in this course.  
Feel free to revisit those notebooks if you need a quick refresher on how to make inference calls or format prompts.

In the next cell, we’re adding a simple example to show how to make an inference call with Groq. This will serve as a quick reference before we start using Groq more extensively in our reasoning frameworks.



In [ ]:
import os

from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Very briefly, explain reasoning frameworks in large language models.",
        }
    ],
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    max_completion_tokens=512
)

print(chat_completion.choices[0].message.content)

### 🛠️ Helper Function: `generate`

To keep our code clean and avoid repeating the same inference boilerplate, we define a small helper function called `generate`.

This function simply sends a prompt to Groq, retrieves the model’s response, and returns the generated text.  
You can call it with a prompt, and optionally adjust the model, temperature, or maximum output length.

We'll use `generate(...)` throughout the notebook to make inference calls concise and readable.


In [ ]:
def generate(
        prompt: str, 
        model: str="meta-llama/llama-4-scout-17b-16e-instruct",
        max_completion_tokens: int=128,
        temperature: float=0.7
    ) -> str:
    
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
        max_completion_tokens=max_completion_tokens,
        temperature=temperature
    )
    return chat_completion.choices[0].message.content

## 🧠 Solving the Game of 24 with LLM Reasoning Frameworks

Now that you’re familiar with the dataset, the evaluation utilities, and our inference engine (Groq), it’s time to start exploring how **large language models** can approach the Game of 24.

Over the next sections, we’ll experiment with several **reasoning frameworks**, structured prompting strategies that help LLMs break down problems, plan steps, and produce more reliable solutions.

### 🔹 First Framework: Few-Shot Prompting

We’ll begin with **few-shot prompting**, where we provide the model with a handful of example puzzles and their solutions.  
These examples act as a guide, helping the model infer the **expected structure** and **reasoning process** before attempting new puzzles on its own.

This will give us a baseline for how well an LLM can solve Game of 24 puzzles when primed with good examples.

Since this is the first and easiet example, we provide the following code.


In [ ]:
io = '''Use numbers and basic arithmetic operations (+ - * /) to obtain 24. Follow the exact format of the following examples and provide just one answer. Do not explain simply list the final expression that reaches 24.

Example: 4 4 6 8
Answer: (4 + 8) * (6 - 4) = 24

Example: 2 9 10 12
Answer: 2 * 12 * (10 - 9) = 24

Example: 4 9 10 13
Answer: (13 - 9) * (10 - 4) = 24

Example: 1 4 8 8
Answer: (8 / 4 + 1) * 8 = 24

Example: 5 5 5 9
Answer: 5 + 5 + 5 + 9 = 24

Input: {input}
'''

state = dataset[0]["Puzzles"]
prompt = io.format(input=state)
response = generate(prompt)
print(response)

### 🔹 Second Framework: Chain of Thought (CoT)

Next, we’ll build on top of the few-shot approach by introducing **Chain of Thought reasoning**.  
While few-shot prompting provides the model with examples of *what* a correct solution looks like, Chain of Thought helps the model understand *how* to get there.

With Chain of Thought, we explicitly ask the model to show its intermediate reasoning steps.  
For the Game of 24, this means the model won’t just output a final expression, it will walk through the arithmetic decisions that lead to the solution.

Adding CoT on top of few-shot prompting will allow us to see whether providing structured reasoning examples improves the model’s accuracy and reliability.


**`TODO:`** Create a new prompt that includes your few-shot examples **plus explicit step-by-step reasoning** for each one, then call Groq and verify the model's output just as you did before.

### 🔹 Third Framework: ReAct (Reason + Act)

After exploring Chain of Thought, we’ll move to a more interactive and modular reasoning framework: **ReAct**.

**ReAct** combines two key capabilities:
- **Reasoning** where the model explains its thought process step by step  
- **Acting** where the model takes actions such as querying tools, updating its internal state, or performing calculations

For the Game of 24, this means the model can:
1. Reason about which numbers to combine  
2. “Act” by generating intermediate expressions  
3. Reflect on whether those actions bring it closer to 24  
4. Continue the cycle until it reaches a final answer

This framework is especially interesting because it simulates an iterative problem-solving process rather than producing the solution in one pass.  

**`TODO:`** Design a prompt that demonstrates the **ReAct** pattern (thinking steps + actions taken). As mentioned above, the action should combine two of the available numbers to produce a new one.

#### 🔍 Parsing Intermediate States in ReAct

Because the **ReAct** framework unfolds step-by-step, we need a way to extract the model’s *current state* after each action it produces.  

Therefore we need to define a helper function that handles this: it reads the model’s action string, identifies whether it’s selecting numbers or producing an expression, and returns the updated state.  


For example consider the input `2 8 8 14`. Imagine the action is the following: `14 - 8 = 6 (left: 2 8 6)`. The current state is `2 8 6`.

**`TODO:`** Complete the `parse_state` function which as the name dictates, parses the state from a given action.

*Note:* There is not a unique solution to this exercise. As you can imagine the parsing function heavily depends on (1) the model you're using and (2) the prompt you have previously defined. The point is that you come up with a prompting + parsing combination that works for you.

In [ ]:
def parse_state(action: str):
    """
    Given an action string from the model, parse and return the current state.
    """
    # TODO: Implement parsing logic based on the action format
    ...

#### The Full ReAct Framework

Now that you have a way to parse intermediate states, you’re ready to implement the complete **ReAct** loop.

Your implementation should:

- Iteratively send the current state back to the model  
- Parse each action using the function from above  
- Update the state at every step  
- Decide when to stop the loop  

While coding, think carefully about:

- **How many steps** the agent should be allowed to take for a 4-number Game of 24 puzzle  
- **What should happen if the current state becomes `24`** before exceeding the step limit  
- **How to detect impossible or looping behavior**  

**`TODO:`** Build a full ReAct solver that progresses step-by-step until it either finds 24 or exhausts its allowed reasoning budget.

### 🔹 Fourth Framework: Tree of Thoughts (ToT)

Now we move to **Tree of Thoughts**, a more powerful reasoning framework that lets the model explore **multiple reasoning paths** rather than committing to a single line of thought.

Tree of Thoughts treats intermediate reasoning steps as **“thought nodes”** in a search tree.  
At each step, the model can propose several possible moves, and we can choose which branches to keep exploring.

#### 🌳 How ToT Works 

For the Game of 24, each “thought” represents one arithmetic operation combining two numbers. A single ToT iteration typically involves:

1. **Generating candidate thoughts**  
   The model proposes several possible intermediate expressions.

2. **Evaluating those thoughts**  
   Each candidate is scored based on how promising it looks (e.g., whether it simplifies the puzzle).

3. **Expanding the search**  
   Promising thoughts become new states, which the model expands in the next round.

#### 🔎 Our Strategy: Breadth-First Search (BFS) ToT

Tree of Thoughts can be implemented in different search styles (DFS, BFS, heuristic search, etc.).  
In this notebook, **we will use the BFS version**, which:

- Expands all thoughts at a given depth before moving deeper  
- Ensures we explore a broad range of possibilities early  
- Helps avoid getting stuck in a poor branch too quickly  
- Works well for relatively small search spaces like a 4-number puzzle

Using BFS gives us a controlled, systematic way to observe how the model reasons when allowed to explore multiple possibilities simultaneously.

---

#### 🔄 Generation & Expansion in ToT-BFS

In the **generation phase**, the model proposes several possible “thoughts” (intermediate moves) for the current puzzle state.  
During the **expansion phase**, each of these thoughts becomes a new node in the search tree, and you repeat the process for all nodes at the current depth.

Because we’re using **BFS**, the algorithm:

- Generates thoughts for *every* node at the current depth  
- Collects all resulting child states  

This ensures a wide exploration of possibilities before diving deeper, helping the solver avoid early mistakes and discover more promising reasoning paths.


**`TODO:`** To make this easier and to spare you from heavy prompt engineering, we provide a **ready-to-use BFS prompt template**. Nevertheless, feel free to experiment with your own adaptations or examine how the prompt interacts under different samples, models or temperatures.

In [ ]:
bfs = '''Use numbers and basic arithmetic operations (+ - * /). Each step, you are only allowed to choose two of the remaining numbers to obtain a new number.  Follow the format of the following examples. Do not explain simply list possible next steps as well as all the remaining numbers and nothing else.

Example: 2 8 8 14
Possible next steps:
2 + 8 = 10 (left: 8 10 14)
8 / 2 = 4 (left: 4 8 14)
14 + 2 = 16 (left: 8 8 16)
2 * 8 = 16 (left: 8 14 16)
8 - 2 = 6 (left: 6 8 14)

Example: 1 3
Possible next steps:
1 + 3 = 4 (left: 4)
1 * 3 = 3 (left: 3)
3 - 1 = 2 (left: 2)
3 / 1 = 3 (left: 3)
1 - 3 = -2 (left: -2)

Input: {input}
Possible next steps:
'''

state = dataset[0]["Puzzles"]
prompt = bfs.format(input=state)
response = generate(prompt)
print(response)

**`TODO:`** The `bfs` prompt suggest multiple next steps in one response. Complete the following function to load all the suggestions in a list of strings.

In [ ]:
def parse_bfs_response(response: str, state: str) -> List[str]:
    """
    Parse the BFS response into a list of proposals.
    If the state is not "24", ensure that each proposal ends with a closing parenthesis.
    """
    # TODO: Implement parsing logic
    ...

proposals = parse_bfs_response(response, state)
print(proposals)

#### 🧮 Evaluation Phase in ToT-BFS

After generating candidate thoughts at the current depth, the **evaluation phase** determines which of them are worth expanding further.

In this phase, the algorithm:

- Assess each thought using a scoring function or heuristic  
- Filter out unpromising or invalid states  
- Keep only the thoughts that appear most promising for the next BFS layer  

The goal is to maintain a manageable search frontier while ensuring that strong reasoning paths continue to propagate through the tree.  
This evaluation step is crucial for guiding the search effectively toward a solution.


**`TODO:`** For the same reasons as before, we provide a **ready-to-use evaluation prompt template**. Nevertheless, feel free to experiment with your own adaptations or examine how the prompt interacts under different samples, models or temperatures.

In [ ]:
evaluation = '''Evaluate if given numbers can reach 24 by responding with the following: "sure", "likely" or "impossible". Follow the format of the following examples. Try to be brief.

Example: 10 14
10 + 14 = 24
sure

Example: 11 12
11 + 12 = 23
12 - 11 = 1
11 * 12 = 132
11 / 12 = 0.91
impossible

Example: 4 4 10
4 + 4 + 10 = 8 + 10 = 18
4 * 10 - 4 = 40 - 4 = 36
(10 - 4) * 4 = 6 * 4 = 24
sure

Example: 4 9 11
9 + 11 + 4 = 20 + 4 = 24
sure

Example: 5 7 8
5 + 7 + 8 = 12 + 8 = 20
(8 - 5) * 7 = 3 * 7 = 21
I cannot obtain 24 now, but numbers are within a reasonable range
likely

Example: 5 6 6
5 + 6 + 6 = 17
(6 - 5) * 6 = 1 * 6 = 6
I cannot obtain 24 now, but numbers are within a reasonable range
likely

Example: 10 10 11
10 + 10 + 11 = 31
(11 - 10) * 10 = 10
10 10 10 are all too big
impossible

Example: 1 3 3
1 * 3 * 3 = 9
(1 + 3) * 3 = 12
1 3 3 are all too small
impossible

Input: {input}
'''

evaluations = []
for proposal in proposals:
    prompt = evaluation.format(input=proposal)
    response = generate(prompt)
    evaluations.append(response)
print(evaluations)

### 🔢 Parsing the Evaluation Scores

During the evaluation step, the model labels each thought with a descriptor indicating how promising it is.  
To make these labels usable in our BFS logic, we **parse** them into numeric scores using:

```python
code_map = {r"impossible": 0.001, r"likely": 1, r"sure": 20}
```

These parsed values let us compare and rank thoughts, helping the BFS process decide which branches should continue to the next level.

**`TODO:`** Complete the following function which parses the evaluations based on the above mapping.

In [ ]:
def parse_evaluation(evaluations: List[str]) -> float:
    """
    Parse evaluation strings into numerical scores.
    Args:
        evaluations (List[str]): List of evaluation strings from the model.
    Returns:
        float: The cumulative score based on the evaluations.

    Why a list of evaluations? People often evaluate the same proposal multiple times to ensure consistency.
    """
    # TODO: Implement parsing logic
    ...

evaluation_scores = [parse_evaluation([evaluation]) for evaluation in evaluations]
evaluation_scores

#### 🌳 Bringing It All Together: Implementing ToT-BFS

You now have all the essential building blocks for **Tree of Thoughts with Breadth-First Search**:

- A structured prompt for generating candidate thoughts  
- A generation & expansion procedure  
- An evaluation phase to score and filter thoughts  
- A parsing step to convert model labels into numeric scores  

Your next step is to **combine these components into a complete ToT-BFS solver**.

As you build it, think about:

- How many thoughts to generate per node  
- How many depth levels the BFS should explore  
- How to maintain and update the frontier of active states  
- When to stop the search (e.g., reaching 24 or exhausting options)

**`TODO:`** Implement the ToT-BFS solver.


### 💡 Hint: Reaching 24 Isn’t the End… Yet

In ToT-BFS, reaching a state valued at **24** means the search has found a *promising path*, but you’re not done.  
You still need to:

- Trace back the sequence of thoughts that led to this state  
- Use those thoughts to **reconstruct the final expression**

Remember: the solver must output a valid arithmetic expression, not just the number 24.  
So once you hit 24, follow the breadcrumb trail of thoughts to build the full solution.


## 🏁 Conclusion

You’ve now explored a full spectrum of **LLM reasoning frameworks**, from simple pattern-based prompting to structured multi-branch search.  
By applying these approaches to the Game of 24, you’ve seen how different reasoning strategies shape the model’s ability to break down problems, plan actions, and arrive at correct solutions.

Each framework offers its own strengths:

- **Few-Shot Prompting** gives the model patterns to mimic.  
- **Chain of Thought** encourages transparent step-by-step reasoning.  
- **ReAct** introduces iterative decision-making with intermediate states.  
- **Tree of Thoughts (BFS)** allows branching exploration and strategic evaluation.

Together, these methods illustrate how prompting and inference design can dramatically influence model behavior—an essential concept for building reliable, controllable AI systems.


### 📚 Summary of Reasoning Frameworks

Below is an expanded overview of the reasoning frameworks covered in this module, plus several influential ones you may want to explore next.  
Each entry includes the **framework name**, its **core idea**, and a **reference** to the original paper or introduction.

| Framework | Core Idea | Link |
|----------|-----------|-----------|
| **Few-Shot Prompting** | Guide the model by showing a handful of solved examples | [https://arxiv.org/abs/2005.14165](https://arxiv.org/abs/2005.14165) |
| **Chain of Thought (CoT)** | Provide step-by-step reasoning to improve multi-step problem solving | [https://arxiv.org/abs/2201.11903](https://arxiv.org/abs/2201.11903) |
| **ReAct** | Interleave explicit reasoning steps with actions taken in an environment |[https://arxiv.org/abs/2210.03629](https://arxiv.org/abs/2210.03629) |
| **Tree of Thoughts (ToT)** | Explore multiple reasoning paths using a structured search tree | [https://arxiv.org/abs/2305.10601](https://arxiv.org/abs/2305.10601) |
| **Reflexion** | An agent learns from failures by reflecting on past attempts to improve future reasoning | [https://arxiv.org/abs/2303.11366](https://arxiv.org/abs/2303.11366) |
| **Language Agent Tree Search (LATS)** | A search algorithm where LLMs guide tree expansion with value and policy evaluations | [https://arxiv.org/abs/2310.04406](https://arxiv.org/abs/2310.04406) |
| **Graph of Thoughts (GoT)** | Represent reasoning as a graph rather than a linear or tree structure, enabling merging and recombination | [https://arxiv.org/abs/2308.09687](https://arxiv.org/abs/2308.09687)|
| **Fleet of Agents (FoA)** | Employs a genetic-type filtering approach to dynamically navigate through the search space | [https://arxiv.org/abs/2405.06691](https://arxiv.org/abs/2405.06691) |

These frameworks highlight the rapidly evolving landscape of LLM reasoning, from simple pattern learning to multi-agent systems and structured search.